In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY") 
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [15]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Virat_Kohli")

In [16]:
docs =loader.load()

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document = text_splitter.split_documents(docs)

In [18]:
document

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Virat_Kohli', 'title': 'Virat Kohli - Wikipedia', 'language': 'en'}, page_content='Virat Kohli - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\

In [19]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="gemma:2b")
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(document,embeddings)


In [25]:
query ="Virt Kohli's batting style and achievements"
result =vectorstore.similarity_search(query)
result[0].page_content



'I love watching Virat Kohli bat. He looks to me like an individual of my own heart. I love his aggression, and he has serious passion that I used to have. He reminds me of myself.—\u200aViv Richards, [56]\nConsistency and #1 rankings (2014–2016)\nDuring the 2014 T20 World Cup, Kohli was named the vice-captain. He made a total of 319 runs at an average of 106.33, recording the most runs by a batter in a single T20 WC tournament.[57] For his consistency, he won the Player of the Tournament award.[58]\nHe topped the T20I batting rankings soon after.[59] Kohli became the fastest batter in the world to go past the 6000-run mark in the ODIs.[60] In December, he was revealed to be the stand-in Test captain.[61] Kohli scored 115, becoming the fourth Indian to score a hundred on Test captaincy debut.[62]'

In [22]:
## Retrival Chain , Document chain 

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following questions based on the context provided to you
<context>{context}</context>
""")
from langchain_ollama import OllamaLLM
llm  = OllamaLLM(model="gemma:2b")
document_chain = create_stuff_documents_chain(llm,prompt)

In [23]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based on the context provided to you\n<context>{context}</context>\n'), additional_kwargs={})])
| OllamaLLM(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [28]:
from langchain_core.documents import Document
document_chain.invoke({"input":query,"context":document})

'**Virat Kohli**\n\n**Languages**\n\n* 51 languages\n\n**Topics**\n\n* Biography\n* Career\n* Cricket career\n* Awards and honors\n* Facts and trivia\n* Controversies\n* Team and rivalries\n* Miscellaneous'

In [30]:
## Retriver --->getting the data from vector store db
retriver=vectorstore.as_retriever()

In [32]:
from langchain.chains import create_retrieval_chain
retriver_chain=create_retrieval_chain(retriver,document_chain)

In [34]:
### GET THE RESPONSE FROM THE llm
response =retriver_chain.invoke({"input":query})

In [35]:
response['answer']

"Sure, here are the answers to the questions you provided:\n\n1. What was Virat Kohli's average in the 2014 T20 World Cup?\nAnswer: 106.33\n\n\n2. What was Virat Kohli's highest score in an ODI match?\nAnswer: 115\n\n\n3. Which country did Virat Kohli play in the 2009–2010 Ranji Trophy season?\nAnswer: Delhi\n\n\n4. What was Virat Kohli's record for most consecutive fifty-plus scores in an ODI match?\nAnswer: 5\n\n\n5. Which team did Virat Kohli lead to victory in the 2019 ODI World Cup?\nAnswer: India"

In [36]:
response

{'input': "Virt Kohli's batting style and achievements",
 'context': [Document(id='7ed2a602-7adc-4621-9d04-dfdf9ea51fc8', metadata={'source': 'https://en.wikipedia.org/wiki/Virat_Kohli', 'title': 'Virat Kohli - Wikipedia', 'language': 'en'}, page_content='I love watching Virat Kohli bat. He looks to me like an individual of my own heart. I love his aggression, and he has serious passion that I used to have. He reminds me of myself.—\u200aViv Richards, [56]\nConsistency and #1 rankings (2014–2016)\nDuring the 2014 T20 World Cup, Kohli was named the vice-captain. He made a total of 319 runs at an average of 106.33, recording the most runs by a batter in a single T20 WC tournament.[57] For his consistency, he won the Player of the Tournament award.[58]\nHe topped the T20I batting rankings soon after.[59] Kohli became the fastest batter in the world to go past the 6000-run mark in the ODIs.[60] In December, he was revealed to be the stand-in Test captain.[61] Kohli scored 115, becoming the

In [7]:
4%4


0